In [ ]:
#pip install antropy

In [2]:
import parselmouth
import numpy as np
import librosa
import nolds
import pandas as pd
import joblib
import antropy as ant

In [30]:
def extract_features(audio_path):
    # Load audio
    sound = parselmouth.Sound(audio_path)
    y, sr = librosa.load(audio_path)

    # 1. MDVP:Fo(Hz), MDVP:Fhi(Hz), MDVP:Flo(Hz)
    pitch = sound.to_pitch()
    fo = pitch.selected_array['frequency']
    MDVP_Fo = np.mean(fo[fo > 0])  # Mean fundamental frequency
    MDVP_Fhi = np.max(fo)           # Max fundamental frequency
    MDVP_Flo = np.min(fo[fo > 0])   # Min fundamental frequency

    # 2. NHR and HNR
    harmonicity = sound.to_harmonicity()
    HNR = harmonicity.values[harmonicity.values != -200].mean()
    NHR = 1 / HNR if HNR != 0 else 0

    # 3. RPDE (Recurrence Period Density Entropy)
    RPDE = ant.perm_entropy(fo, normalize=True)    # or use a different signal for better accuracy

    # 4. DFA (Detrended Fluctuation Analysis)
    DFA = nolds.dfa(y)

    # 5. Spread2
    spread = librosa.feature.spectral_bandwidth(y=y, sr=sr)
    Spread2 = spread.mean() / (sr / 2)

    # 6. D2 (Correlation Dimension)
    D2 = nolds.corr_dim(fo, emb_dim=25)  # Adjust emb_dim for accuracy, perhaps check paper

    # 7. PPE (Pitch Period Entropy)
    PPE = ant.app_entropy(fo)

    # Consolidate into dictionary
    features = {
        "MDVP:Fo(Hz)": MDVP_Fo,
        "MDVP:Fhi(Hz)": MDVP_Fhi,
        "MDVP:Flo(Hz)": MDVP_Flo,
        "NHR": NHR,
        "HNR": HNR,
        "RPDE": RPDE,
        "DFA": DFA,
        "Spread2": Spread2,
        "D2": D2,
        "PPE": PPE
    }
    
    return features

In [31]:
audio_file = '../data/raw/Mum.wav'
features = extract_features(audio_file)
print(features)

{'MDVP:Fo(Hz)': 170.6526896754848, 'MDVP:Fhi(Hz)': 212.21061983952993, 'MDVP:Flo(Hz)': 164.8448531704857, 'NHR': 0.05662130007900399, 'HNR': 17.661198146363557, 'RPDE': 0.8160944289032286, 'DFA': 0.019018089128222004, 'Spread2': 0.1589928748264482, 'D2': 1.5470174175686484, 'PPE': 0.023943007147128936}


In [32]:
feature_df = pd.DataFrame(columns=[
    "MDVP:Fo(Hz)", "MDVP:Fhi(Hz)", "MDVP:Flo(Hz)", "NHR", "HNR", 
    "RPDE", "DFA", "Spread2", "D2", "PPE"
])

In [33]:
feature_df = pd.concat([feature_df, pd.DataFrame([features])], ignore_index=True)

C:\Users\bert_\AppData\Local\Temp\ipykernel_32752\1364591689.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  feature_df = pd.concat([feature_df, pd.DataFrame([features])], ignore_index=True)


In [34]:
feature_df.head()

,MDVP:Fo(Hz),MDVP:Fhi(Hz),MDVP:Flo(Hz),NHR,HNR,RPDE,DFA,Spread2,D2,PPE
0,170.65269,212.21062,164.844853,0.056621,17.661198,0.816094,0.019018,0.158993,1.547017,0.023943


In [35]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df_scaled = scaler.fit_transform(feature_df)

In [36]:
test_model = joblib.load("../models/Test3_model.joblib")

In [37]:
prediction = test_model.predict(df_scaled)

In [38]:
print("Predictions:", prediction)

Predictions: [1]
